In [246]:
# reading in fasta sequence
with open('input.txt', 'r') as raw_fasta:
    seq = ''.join(raw_fasta.read().splitlines()[1:])

print(seq)

AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG


In [247]:
# get reverse complement for later
complement_dict = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
reverse_complement = ''.join([complement_dict[nuc] for nuc in seq][::-1])
print(reverse_complement)


CTGAGATGCTACTCGGATCATTCAGGCTTATTCCAAAAGAGACTCTAATCCAAGTCGCGGGGTCATCCCCATGTAACCTGAGTTAGCTACATGGCT


In [248]:
# generate all 6 reading frames(3 from original string, 3 from reverse complement)

rf_original = [seq[i:] for i in range(3)]
rf_reverse_complement = [reverse_complement[i:] for i in range(3)]
rf = rf_original + rf_reverse_complement
print(rf)

['AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG', 'GCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG', 'CCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG', 'CTGAGATGCTACTCGGATCATTCAGGCTTATTCCAAAAGAGACTCTAATCCAAGTCGCGGGGTCATCCCCATGTAACCTGAGTTAGCTACATGGCT', 'TGAGATGCTACTCGGATCATTCAGGCTTATTCCAAAAGAGACTCTAATCCAAGTCGCGGGGTCATCCCCATGTAACCTGAGTTAGCTACATGGCT', 'GAGATGCTACTCGGATCATTCAGGCTTATTCCAAAAGAGACTCTAATCCAAGTCGCGGGGTCATCCCCATGTAACCTGAGTTAGCTACATGGCT']


In [249]:
# find ORF of one reading frame.
import re
rf_codons = {}
for ind, rf_indiv in enumerate(rf):
    rf_codons[ind + 1] = [rf_indiv[i:i + 3] for i in range(0,(len(rf_indiv) // 3) * 3, 3)]
print(rf_codons)

{1: ['AGC', 'CAT', 'GTA', 'GCT', 'AAC', 'TCA', 'GGT', 'TAC', 'ATG', 'GGG', 'ATG', 'ACC', 'CCG', 'CGA', 'CTT', 'GGA', 'TTA', 'GAG', 'TCT', 'CTT', 'TTG', 'GAA', 'TAA', 'GCC', 'TGA', 'ATG', 'ATC', 'CGA', 'GTA', 'GCA', 'TCT', 'CAG'], 2: ['GCC', 'ATG', 'TAG', 'CTA', 'ACT', 'CAG', 'GTT', 'ACA', 'TGG', 'GGA', 'TGA', 'CCC', 'CGC', 'GAC', 'TTG', 'GAT', 'TAG', 'AGT', 'CTC', 'TTT', 'TGG', 'AAT', 'AAG', 'CCT', 'GAA', 'TGA', 'TCC', 'GAG', 'TAG', 'CAT', 'CTC'], 3: ['CCA', 'TGT', 'AGC', 'TAA', 'CTC', 'AGG', 'TTA', 'CAT', 'GGG', 'GAT', 'GAC', 'CCC', 'GCG', 'ACT', 'TGG', 'ATT', 'AGA', 'GTC', 'TCT', 'TTT', 'GGA', 'ATA', 'AGC', 'CTG', 'AAT', 'GAT', 'CCG', 'AGT', 'AGC', 'ATC', 'TCA'], 4: ['CTG', 'AGA', 'TGC', 'TAC', 'TCG', 'GAT', 'CAT', 'TCA', 'GGC', 'TTA', 'TTC', 'CAA', 'AAG', 'AGA', 'CTC', 'TAA', 'TCC', 'AAG', 'TCG', 'CGG', 'GGT', 'CAT', 'CCC', 'CAT', 'GTA', 'ACC', 'TGA', 'GTT', 'AGC', 'TAC', 'ATG', 'GCT'], 5: ['TGA', 'GAT', 'GCT', 'ACT', 'CGG', 'ATC', 'ATT', 'CAG', 'GCT', 'TAT', 'TCC', 'AAA', 'AGA', 'G

In [57]:
# testing making a json
import json

with open('json_test', 'w') as write_file:
    json.dump(rf_codons, write_file)

In [33]:
# find all indices of 'ATG' start codon
import numpy as np
start_stop_dict = {}
for ind, codons in rf_codons.items():
    start_stop_dict[ind] = [np.array([index for index, codon in enumerate(codons) if codon == 'ATG']), np.array([index for index, codon in enumerate(codons) if codon in ('TGA', 'TAA', 'TAG')])]

print(start_stop_dict)

{1: [array([ 8, 10, 25]), array([22, 24])], 2: [array([1]), array([ 2, 10, 16, 25, 28])], 3: [array([], dtype=float64), array([3])], 4: [array([30]), array([15, 26])], 5: [array([23]), array([ 0, 24])], 6: [array([1]), array([27])]}


2 conditions
1. start or stop codons codons empty -> no operations needed
2. at least 1 element in stop and start codon indices.



In [40]:
np.array([1,2,3]) - 3

array([-2, -1,  0])

In [102]:
# correct algorithm for finding all indexes of all ORF's for one total reading frame
# output is a list of lists ex [8,22], []
test = start_stop_dict[1]
# print(test[0])
if (test[0].size > 0) and (test[1].size > 0): # check there are no empty inds in either start or stop inds.
    # each start codon may or may not have an ORF, the maximum # of ORF's per frame = # of start codons in frame.
    # if this calculation contains a + value, then there is an ORF. (test[1] - test[0][0:len(test[0])])

    # get the stop codons minus each start codon, creates a list of arrays, each array corresponds to one start codon subtraction from all stop codons
    stop_minus_start = [test[1] - test[0][i] for i in range(len(test[0]))]
    print(stop_minus_start)
    # if the list has more than one positive value, the index of the lowest + val corresponds to the index of the stop codon at the end of the ORF.
    



[array([14, 16]), array([12, 14]), array([-3, -1])]


In [150]:
# get a list to determine what start codons should be evaluated. (if differences are all negative then there is no possible ORF)

diff_lengths = np.array([len(np.zeros(len(stop_minus_start[0]))[stop_minus_start[i] > 0]) for i in range(len(stop_minus_start))])
print(diff_lengths)
print(diff_lengths > 0) # get logical array of possible start codon position.

[2 2 0]
[ True  True False]


In [151]:
# get the indexes of the start codons to be evalutated.

possible_start_inds = start_stop_dict[1][0][diff_lengths > 0]
print(possible_start_inds)

[ 8 10]


In [152]:
# diff_lengths[1] = 0
# diff_lengths[2] = 1
# find the inds of the 
# getting the correct indexes for stop_minus_start based on which ones are valid.
placeholder_inds = np.arange(len(diff_lengths))
print(placeholder_inds)
start_codon_inds = placeholder_inds[diff_lengths > 0]
print(start_codon_inds)
# np.arange(len(diff_lengths))[diff_lengths > 0]


[0 1 2]
[0 1]


In [245]:
# get a list of corresponding length, of the stop codons to be evaluating.
# for each possible start codon, the stop codon could be different.
print(start_stop_dict[1][1]) # gets the list of stop codons for a particular reading frame [1] in this case.
print(stop_minus_start[0] > 0) # this is testing the first start codon b/c [0]. gets logical of valid inds for stop codons.append
print(start_stop_dict[1][1][stop_minus_start[0] > 0][0]) # gets the first positive difference corresponding the the ORF stop codon for a particular start codon.
possible_end_inds = [start_stop_dict[1][1][stop_minus_start[i] > 0][0] for i in start_codon_inds]
print(possible_end_inds)


[22 24]
[ True]


IndexError: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 1

In [167]:
# determine which stop codon is the end of the ORF
# the logical follows that the first non negative difference holds the index of the stop codon
test2 = stop_minus_start[0] # [14,16]

start_stop_dict[1][1][test2 > 0][0] # np.array([22,24])[True, True][0] = 22

22

In [173]:
# this gives a list with pairs as ORF's in one reading frame.
orfs_1frame = list(zip(possible_start_inds, possible_end_inds))
print(orfs_1frame)
print(orfs_1frame[0][0])

[(8, 22), (10, 22)]
8


In [251]:
import numpy as np
# reading in fasta
with open('input.txt', 'r') as raw_fasta:
    seq = ''.join(raw_fasta.read().splitlines()[1:])

# getting codon dictionary with codons as keys and corresponding amino acids as values
with open('dna_codon_table.txt', 'r') as f_codon:
    codon_table = f_codon.read().split()
    codon_dict = {codon_table[z]: codon_table[z+1] for z in range(0, len(codon_table), 2)}

# getting reverse complement
complement_dict = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
reverse_complement = ''.join([complement_dict[nuc] for nuc in seq][::-1])

# get all 6 reading frames
rf_original = [seq[i:] for i in range(3)]
rf_reverse_complement = [reverse_complement[i:] for i in range(3)]
rf = rf_original + rf_reverse_complement

# putting all reading frames into a dict
rf_codons = {}
for ind, rf_indiv in enumerate(rf):
    rf_codons[ind + 1] = [rf_indiv[i:i + 3] for i in range(0,(len(rf_indiv) // 3) * 3, 3)]

# getting all start and stop codon positions for each reading frame
start_stop_dict = {}
for ind, codons in rf_codons.items():
    start_stop_dict[ind] = [np.array([index for index, codon in enumerate(codons) if codon == 'ATG']), np.array([index for index, codon in enumerate(codons) if codon in ('TGA', 'TAA', 'TAG')])]


# get dictionary with reading frame as key and all ORF's within a frame as values. (orfs_frame_dict)
orfs_frame_dict = {}
for frame_num, start_stop_dict_frame in start_stop_dict.items(): 
    # get all ORFs for all 6 reading frames.
    if (start_stop_dict_frame[0].size > 0) and (start_stop_dict_frame[1].size > 0): # check there are no empty inds in either start or stop inds.
        
        # get list of stop codon list minus each start codon 
        stop_minus_start = [start_stop_dict_frame[1] - start_stop_dict_frame[0][i] for i in range(len(start_stop_dict_frame[0]))]
                    
        # get lengths of logical arrays that tell how many differences for each start codon are positive(meaning they are possible)
        diff_lengths = np.array([len(np.zeros(len(stop_minus_start[0]))[stop_minus_start[i] > 0]) for i in range(len(stop_minus_start))])

        # get the inds of the start codons that are in a working ORF. (these inds correspond to the ones in rf_condons[x][:])
        possible_start_inds = start_stop_dict_frame[0][diff_lengths > 0]  
        if np.any(possible_start_inds): # check that there are possible start inds
        # get the inds of the start codons that are in a working ORF (these inds correspond to the ones in stop_minus_start[x])
            start_codon_inds = np.arange(len(diff_lengths))[diff_lengths > 0]
            possible_end_inds = [start_stop_dict_frame[1][stop_minus_start[i] > 0][0] for i in start_codon_inds]

            orfs_frame_dict[frame_num] = list(zip(possible_start_inds, possible_end_inds))

# print(start_stop_dict)
# print(orfs_frame_dict)

# get all codon strings
codon_strings = []
for k,v in orfs_frame_dict.items():
    # print(k,len(v))
    for i in range(len(v)):
        codon_strings.append(rf_codons[k][v[i][0]: v[i][1]])

# translate codon ORFS into amino acids sequences. put them in a set to avoid repeats.
proteins = set()
for codon_seq in codon_strings:
    proteins.add(''.join([codon_dict[codon] for codon in codon_seq]))

for protein in proteins:
    print(protein) 


MTPRLGLESLLE
MLLGSFRLIPKETLIQVAGSSPCNLS
MGMTPRLGLESLLE
M


In [199]:
len(orfs_frame_dict)
orfs_frame_dict[1][0][1]

22

In [202]:
# get all codon strings
codon_strings = []
for k,v in orfs_frame_dict.items():
    # print(k,len(v))
    for i in range(len(v)):
        codon_strings.append(rf_codons[k][v[i][0]: v[i][1]])

print(codon_strings)

[['ATG', 'GGG', 'ATG', 'ACC', 'CCG', 'CGA', 'CTT', 'GGA', 'TTA', 'GAG', 'TCT', 'CTT', 'TTG', 'GAA'], ['ATG', 'ACC', 'CCG', 'CGA', 'CTT', 'GGA', 'TTA', 'GAG', 'TCT', 'CTT', 'TTG', 'GAA'], ['ATG'], ['ATG'], ['ATG', 'CTA', 'CTC', 'GGA', 'TCA', 'TTC', 'AGG', 'CTT', 'ATT', 'CCA', 'AAA', 'GAG', 'ACT', 'CTA', 'ATC', 'CAA', 'GTC', 'GCG', 'GGG', 'TCA', 'TCC', 'CCA', 'TGT', 'AAC', 'CTG', 'AGT']]


In [238]:
# translate codon strings into proteins
with open('dna_codon_table.txt', 'r') as f_codon:
    codon_table = f_codon.read().split()
    codon_dict = {codon_table[z]: codon_table[z+1] for z in range(0, len(codon_table), 2)}
    
print(codon_dict)

{'TTT': 'F', 'CTT': 'L', 'ATT': 'I', 'GTT': 'V', 'TTC': 'F', 'CTC': 'L', 'ATC': 'I', 'GTC': 'V', 'TTA': 'L', 'CTA': 'L', 'ATA': 'I', 'GTA': 'V', 'TTG': 'L', 'CTG': 'L', 'ATG': 'M', 'GTG': 'V', 'TCT': 'S', 'CCT': 'P', 'ACT': 'T', 'GCT': 'A', 'TCC': 'S', 'CCC': 'P', 'ACC': 'T', 'GCC': 'A', 'TCA': 'S', 'CCA': 'P', 'ACA': 'T', 'GCA': 'A', 'TCG': 'S', 'CCG': 'P', 'ACG': 'T', 'GCG': 'A', 'TAT': 'Y', 'CAT': 'H', 'AAT': 'N', 'GAT': 'D', 'TAC': 'Y', 'CAC': 'H', 'AAC': 'N', 'GAC': 'D', 'TAA': 'Stop', 'CAA': 'Q', 'AAA': 'K', 'GAA': 'E', 'TAG': 'Stop', 'CAG': 'Q', 'AAG': 'K', 'GAG': 'E', 'TGT': 'C', 'CGT': 'R', 'AGT': 'S', 'GGT': 'G', 'TGC': 'C', 'CGC': 'R', 'AGC': 'S', 'GGC': 'G', 'TGA': 'Stop', 'CGA': 'R', 'AGA': 'R', 'GGA': 'G', 'TGG': 'W', 'CGG': 'R', 'AGG': 'R', 'GGG': 'G'}


In [232]:
# translate codon ORFS into amino acids sequences. put them in a set to avoid repeats.
proteins = set()
for codon_seq in codon_strings:
    proteins.add(''.join([codon_dict[codon] for codon in codon_seq]))

for protein in proteins:
    print(protein) 


MTPRLGLESLLE
MLLGSFRLIPKETLIQVAGSSPCNLS
MGMTPRLGLESLLE
M


In [5]:
# how to import a function from a python file in another directory
import sys
# print(sys.path)
sys.path.insert(1, 'c:\\Users\\nate\\PycharmProjects\\rosalind')
print(sys.path)




['c:\\Users\\nate\\PycharmProjects\\rosalind\\bioinformatics\\Open Reading Frames', 'c:\\Users\\nate\\PycharmProjects\\rosalind', 'c:\\Users\\nate\\PycharmProjects\\rosalind\\bioinformatics', 'c:\\Users\\nate\\PycharmProjects\\rosalind\\bioinformatics', 'C:\\Users\\nate\\anaconda3\\python39.zip', 'C:\\Users\\nate\\anaconda3\\DLLs', 'C:\\Users\\nate\\anaconda3\\lib', 'C:\\Users\\nate\\anaconda3', '', 'C:\\Users\\nate\\anaconda3\\lib\\site-packages', 'C:\\Users\\nate\\anaconda3\\lib\\site-packages\\locket-0.2.1-py3.9.egg', 'C:\\Users\\nate\\anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\nate\\anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\nate\\anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\nate\\anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\nate\\.ipython', 'C:\\Users\\nate\\AppData\\Roaming\\Python\\Python39\\site-packages']


In [7]:
from rosalind import amino_combo_dict

# learn from other solutions


In [281]:
from Bio.Seq import Seq
import re

seq = Seq("AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG")

orfs = set([])
prot = re.compile("(?=(M.*?\*))")
for frame in range(3):
    orfs = orfs.union(set(prot.findall(str(seq[frame:(len(seq[frame:]) // 3) * 3].translate()))))
    orfs = orfs.union(set(prot.findall(str(seq.reverse_complement()[frame:(len(seq[frame:]) // 3) * 3].translate()))))

for i in orfs:
    print(i[0:-1])

MTPRLGLESLLE
M
MLLGSFRLIPKETLIQVAGSSPCNLS
MGMTPRLGLESLLE


In [255]:
seq = Seq("AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG")
print(seq.translate())

SHVANSGYMGMTPRLGLESLLE*A*MIRVASQ


In [263]:
test = set()
test.add(1)
print(test)
test2 = set([1,2])
test2.un

{1}


{1, 2}

### For loop based approach instead of reg expression module approach
###### side note: shift + l makes the line numberings dissappear

In [6]:
import string
def revcomp(s):
    r = s.translate(s.maketrans('ACTG','TGAC'))
    return r[::-1]

dna = "AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG"

table = """TTT F      CTT L      ATT I      GTT V
TTC F      CTC L      ATC I      GTC V
TTA L      CTA L      ATA I      GTA V
TTG L      CTG L      ATG M      GTG V
TCT S      CCT P      ACT T      GCT A
TCC S      CCC P      ACC T      GCC A
TCA S      CCA P      ACA T      GCA A
TCG S      CCG P      ACG T      GCG A
TAT Y      CAT H      AAT N      GAT D
TAC Y      CAC H      AAC N      GAC D
TAA Stop   CAA Q      AAA K      GAA E
TAG Stop   CAG Q      AAG K      GAG E
TGT C      CGT R      AGT S      GGT G
TGC C      CGC R      AGC S      GGC G
TGA Stop   CGA R      AGA R      GGA G
TGG W      CGG R      AGG R      GGG G"""
table = dict(zip(table.split()[::2],table.split()[1::2]))

stop = ["TAA","TAG","TGA"]
proteins = []
for s in [dna,revcomp(dna)]:
    for i in range(len(s)):
        if s[i:i+3] == "ATG":
            for j in range(i,len(s),3):
                if s[j:j+3] in stop:
                    c = [s[k:k+3] for k in range(i,j,3)]
                    proteins.append( ''.join(map(lambda x:table[x],c)) )
                    break
for seq in set(proteins):
    print(seq)

M
MGMTPRLGLESLLE
MTPRLGLESLLE
MLLGSFRLIPKETLIQVAGSSPCNLS


In [4]:
s = 'ATTCGG'
# s.maketrans('ACTG','TGAC')
s.translate(s.maketrans('ACTG','TGAC'))

'TAAGCC'